<a href="https://colab.research.google.com/github/MelissaLJ/Consumer_Complaints_Analysis-/blob/master/homework_chapter_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install cython


In [2]:
%load_ext Cython

In [3]:
import numpy as np
import pandas as pd
import time

In [4]:
# 函数1
def target_mean_v1(data,y,x):
  result = np.zeros(data.shape[0])
  for i in range(data.shape[0]):
    groupby_result = data[data.index != i].groupby([x],as_index=False).agg(['mean','count'])
    result[i] = groupby_result.loc[groupby_result.index == data.loc[i,x],(y,'mean')]

In [5]:
# 函数2
def target_mean_v2(data,y,x):
  result = np.zeros(data.shape[0])
  value_dict = dict()
  count_dict = dict()
  for i in range(data.shape[0]):
    if data.loc[i,x] not in value_dict.keys():
      value_dict[data.loc[i,x]] = data.loc[i,y]
      count_dict[data.loc[i,x]] = 1
    else:
      value_dict[data.loc[i,x]] += data.loc[i,y]
      count_dict[data.loc[i,x]] += 1
  for i in range(data.shape[0]):
      result[i] = (value_dict[data.loc[i, x]] - data.loc[i, y]) / (count_dict[data.loc[i,x]] - 1)
  return result    

In [6]:
# 函数3
%%cython
import numpy as np
cimport numpy as np

cpdef target_mean_v3(data,y_name,x_name):
  cdef long nrows = data.shape[0]
  cdef np.ndarray[double] result = np.asfortranarray(np.zeros(nrows), dtype=np.float64)
  cdef np.ndarray[double] y = np.asfortranarray(data[y_name], dtype=np.float64)
  cdef np.ndarray[double] x = np.asfortranarray(data[x_name], dtype=np.float64)
  target_mean_v3_func(result,y,x,nrows)
  return result

cdef target_mean_v3_func(double[:] result, double[:] y, double[:] x, const long nrows):
  cdef dict value_dict = dict()
  cdef dict count_dict = dict()
  cdef long i=0
  for i in range(nrows):
    if x[i] not in value_dict.keys():
      value_dict[x[i]] = y[i]
      count_dict[x[i]] = 1
    else:
      value_dict[x[i]] += y[i]
      count_dict[x[i]] += 1
  for i in range(nrows):
    result[i] = (value_dict[x[i]] - y[i]) / (count_dict[x[i]] - 1)
  return result
  

In [7]:
# 函数4
%%cython --cplus 
from libcpp.unordered_map cimport unordered_map 
import numpy as np
cimport numpy as np


cpdef target_mean_v4(data,y_name,x_name):
  cdef long nrows = data.shape[0]
  cdef np.ndarray[double] result = np.asfortranarray(np.zeros(nrows), dtype=np.float)
  cdef np.ndarray[double] y = np.asfortranarray(data[y_name], dtype=np.float)
  cdef np.ndarray[long] x = np.asfortranarray(data[x_name], dtype=np.int)
  target_mean_v4_func(result,y,x,nrows)
  return result

cdef target_mean_v4_func(double[:] result, double[:] y, long[:] x, const long nrows):
  cdef unordered_map[long, double] value_map 
  cdef unordered_map[long, double] count_map 
  cdef long i=0
  for i from 0 <= i < nrows by 1:
    if not value_map.count(x[i]):
      value_map[x[i]] = y[i]
      count_map[x[i]] = 1
    else:
      value_map[x[i]] += y[i]
      count_map[x[i]] += 1
  for i from 0 <= i < nrows by 1:
    result[i] = (value_map[x[i]] - y[i]) / (count_map[x[i]] - 1)
  return result

In [8]:
# 函数5
%%cython --cplus 
from libcpp.unordered_map cimport unordered_map 
import numpy as np
cimport numpy as np


cpdef target_mean_v5(data,y_name,x_name):
  cdef long nrows = data.shape[0]
  cdef np.ndarray[double] result = np.asfortranarray(np.zeros(nrows), dtype=np.float)
  cdef np.ndarray[double] y = np.asfortranarray(data[y_name], dtype=np.float)
  cdef np.ndarray[long] x = np.asfortranarray(data[x_name], dtype=np.int)
  target_mean_v5_func(result,y,x,nrows)
  return result

cdef target_mean_v5_func(double[:] result, double[:] y, long[:] x, const long nrows):
  cdef unordered_map[long, double] value_map 
  cdef unordered_map[long, double] count_map 
  cdef:
    unordered_map[long, double].iterator end = value_map.end()
  for i from 0 <= i < nrows by 1:
    if value_map.find(x[i]) == end:
      value_map[x[i]] = y[i]
      count_map[x[i]] = 1
    else:
      value_map[x[i]] += y[i]
      count_map[x[i]] += 1
  for i from 0 <= i < nrows by 1:
    result[i] = (value_map[x[i]] - y[i]) / (count_map[x[i]] - 1)
  return result

In [9]:
def main():
    y = np.random.randint(2, size=(5000, 1))
    x = np.random.randint(10, size=(5000, 1))
    data = pd.DataFrame(np.concatenate([y, x], axis=1), columns=['y', 'x'])

    start_1 = time.time()
    result_1 = target_mean_v1(data, 'y', 'x') 
    end_1 = time.time()
    print('The runtime for the 1st function is',end_1 - start_1)

    start_2 = time.time()
    result_2 = target_mean_v2(data, 'y', 'x') # 
    end_2 = time.time()
    print('The runtime for the 2nd function is',end_2 - start_2)

    start_3 = time.time()
    result_3 = target_mean_v3(data, 'y', 'x') # 
    end_3 = time.time()
    print('The runtime for the 3rd function is',end_3 - start_3)

    start_4 = time.time()
    result_4 = target_mean_v4(data, 'y', 'x')
    end_4 = time.time()
    print('The runtime for the 4th function is',end_4 - start_4)

    start_5 = time.time()
    result_5 = target_mean_v5(data, 'y', 'x')
    end_5 = time.time()
    print('The runtime for the 5th function is',end_5 - start_5)

if __name__ == '__main__':
    main()

The runtime for the 1st function is 27.321754455566406
The runtime for the 2nd function is 0.3227241039276123
The runtime for the 3rd function is 0.0028243064880371094
The runtime for the 4th function is 0.0007383823394775391
The runtime for the 5th function is 0.0006363391876220703
